In [ ]:
#| default_exp feature.users

# Users

- [ ] - get_users() route implemented without loop.  default limit is 50. 

In [ ]:
#| exporti

from dataclasses import dataclass , field

import mbison.client.core as dmda

In [ ]:
#| hide
import nbdev
import os

In [ ]:
auth = dmda.DomoAuth(
    domo_instance=os.environ["DOMO_INSTANCE"],
    access_token=os.environ["DOMO_ACCESS_TOKEN"],
)
# auth.who_am_i()
auth

*** Using Access Token Auth


DomoAuth(domo_instance='domo-community', username=None)

In [ ]:
#| export
class User_API_Exception(dmda.API_Exception):
    def __init__(self, res, message= None):

        super().__init__(res = res,message = message)

In [ ]:
#| export

def get_users(auth : dmda.DomoAuth , debug_api: bool = False):

    endpoint = f"/api/content/v3/users/"

    params = {"includeDetails": True}

    res = dmda.domo_api_request(
        auth= auth,
        endpoint=endpoint,
        request_type= "GET",
        params = params,
        debug_api= debug_api,
    )


    if not res.is_success and res.status == 404 :
        raise User_API_Exception(res, message = f"unable to find user - {user_id}")

    return res

In [ ]:
res =get_users( auth =auth, debug_api = False)

print(len(res.response))
user = res.response[0]
user

50


{'id': 1216550715,
 'invitorUserId': 1893952720,
 'displayName': '8:26 - go to sleep',
 'role': 'Privileged',
 'roleId': 2,
 'detail': {'email': 'test4@domo.com',
  'phoneNumber': '152',
  'pending': True,
  'active': True,
  'created': 1664938821,
  'modified': 1718758313,
  'department': 'test'}}

In [ ]:
#| export

def get_user_by_id(user_id, auth : dmda.DomoAuth , debug_api: bool = False):

    endpoint = f"/api/content/v3/users/{user_id}"

    params = {"includeDetails": True}

    res = dmda.domo_api_request(
        auth= auth,
        endpoint=endpoint,
        request_type= "GET",
        params = params,
        debug_api= debug_api,
    )


    if not res.is_success and res.status == 404 :
        raise User_API_Exception(res, message = f"unable to find user - {user_id}")

    return res

In [ ]:
(get_user_by_id(user_id = user['id'], auth =auth, debug_api = False)).response


{'id': 1216550715,
 'invitorUserId': 1893952720,
 'displayName': '8:26 - go to sleep',
 'role': 'Privileged',
 'roleId': 2,
 'detail': {'email': 'test4@domo.com',
  'phoneNumber': '152',
  'pending': True,
  'active': True,
  'created': 1664938821,
  'modified': 1718758313,
  'department': 'test'}}

In [ ]:
#| export

@dataclass
class DomoUser:
    auth :dmda.DomoAuth = field(repr = False)
    id: str
    display_name: str
    role_id: int
    email : str

    @classmethod
    def from_json(cls, obj, auth : dmda.DomoAuth):
        return cls(
            auth = auth, 
            id = obj['id'],
            display_name = obj['displayName'],
            role_id = obj['roleId'],
            email = obj['detail']['email']
        )
    
    @classmethod
    def get_by_id(cls, user_id, auth, debug_api : bool = False, return_raw :bool = False):
        res = get_user_by_id(user_id = user_id, auth =auth, debug_api = debug_api)

        if return_raw:
            return res
        
        return cls.from_json(obj = res.response , auth = auth)

In [ ]:
DomoUser.get_by_id(user_id= user['id'], auth = auth)


DomoUser(id=1216550715, display_name='8:26 - go to sleep', role_id=2, email='test4@domo.com')

In [ ]:
#| hide


nbdev.nbdev_export()